In [20]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


C:\Python39\lib\site-packages\IPython\core\magics\pylab.py:159: UserWarning: pylab import has clobbered these variables: ['norm']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [21]:
import numpy as np
import mnist
from tensorflow import keras
from PIL import Image
from PIL import ImageEnhance

# The first time you run this might be a bit slow, since the
# mnist package has to download and cache the data.
train_images = mnist.train_images()
train_labels = mnist.train_labels()

print(train_images.shape) # (60000, 28, 28)
print(train_labels.shape) # (60000,)

# gradient between 0 and 1 for 256*256
array = np.linspace(0,1,256*256)

# reshape to 2d
#mat = np.reshape(train_images[1],(256,256))

# Creates PIL image
img = Image.fromarray(train_images[1], "L")
#new_image = img.resize((400,400))
enhancer2 = ImageEnhance.Contrast(img)
new_image = enhancer2.enhance(1)
enhancer1 = ImageEnhance.Sharpness(new_image)
new_image = enhancer1.enhance(100)
new_image.show()

(60000, 28, 28)
(60000,)


In [22]:
import numpy as np
import mnist

train_images = mnist.train_images()
train_labels = mnist.train_labels()
test_images = mnist.test_images()
test_labels = mnist.test_labels()

# Normalize the images.
train_images = (train_images / 255)
test_images = (test_images / 255)

# Reshape the images.
train_images = np.expand_dims(train_images, axis=3)
test_images = np.expand_dims(test_images, axis=3)

print(train_images.shape) # (60000, 28, 28, 1)
print(test_images.shape)  # (10000, 28, 28, 1)

(60000, 28, 28, 1)
(10000, 28, 28, 1)


In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from tensorflow.keras.utils import to_categorical

num_filters = 8
filter_size = 3
pool_size = 2

model = Sequential([
  Conv2D(num_filters, filter_size, input_shape=(28, 28, 1), strides=2, padding='same', activation='relu'),
  MaxPooling2D(pool_size=pool_size),
  Flatten(),
  Dense(10, activation='softmax'),
])
# Compile the model.
model.compile(
  'adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)

# Train the model.
model.fit(
  train_images,
  to_categorical(train_labels),
  epochs=3,
  validation_data=(test_images, to_categorical(test_labels)),
)
model.save_weights('cnn.h5')

Epoch 1/3
1875/1875 [==============================] - 5s 2ms/step - loss: 0.4792 - accuracy: 0.8600 - val_loss: 0.2730 - val_accuracy: 0.9176
Epoch 2/3
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2428 - accuracy: 0.9278 - val_loss: 0.2004 - val_accuracy: 0.9423
Epoch 3/3
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1983 - accuracy: 0.9415 - val_loss: 0.1855 - val_accuracy: 0.9464


In [13]:
model.load_weights('cnn.h5')

# Predict on the first 5 test images.
predictions = model.predict(test_images)

# Print our model's predictions.
#print(np.argmax(predictions, axis=1)) # [7, 2, 1, 0, 4]

# Check our predictions against the ground truths.
#print(test_labels) # [7, 2, 1, 0, 4]
sum = 0
#for i in range(len(predictions)):
    #if (predictions[i] == test_labels[i]).all():
        #sum += 1
#sum / len(predictions)


NameError: name 'model' is not defined

In [17]:
an_array = np.random.rand(10)*10
print(an_array)

norm = np.linalg.norm(an_array)
print(norm)
normal_array = an_array/norm
print(normal_array)

[2.03994112 9.62592623 0.9862387  2.88660973 2.57920283 9.15633371
 0.29481945 7.83192071 8.26013099 4.04754924]
18.51091709546645
[0.11020206 0.52001347 0.05327876 0.15594094 0.13933415 0.49464506
 0.01592679 0.42309739 0.44623024 0.21865741]


In [13]:
def homogeneous(X):
    return np.insert(X, 0, values=1, axis=-1)

class Layer:
    def __init__(self, input_size=None, output_size=None):
        self.input_size = input_size
        self.output_size = output_size
        self.prev = None
        self.next = None
    def __call__(self, callable_graph):
        self.prev = callable_graph
        callable_graph.next = self
        self.input_size = callable_graph.output_size
        if self.output_size is None:
            self.output_size = self.input_size
        
        return self
    
    def predict(self, X):
        return self.forward(X) if self.next is None else self.next.predict(self.forward(X))
    
    def bakcdrop(self, Y):
        return self.backward(Y) if self.prev is None else self.prev.backdrop(self.backward(Y))

    def str_chain(self):
        return str(self) if self.next is None else str(self) + " -> " + self.next.str_chain()
    
class Dense(Layer):
    def __init__(self, units, input_size=None):
        Layer.__init__(self, input_size, units)
        self.W = None
    
    def init_weights(self):
        self.W = np.random.normal(0, scale=2/(self.input_size), size=(self.output_size, self.input_size + 1))
    
    def forward(self, X):
        if self.W is None:
            self.init_weights()
        
        Xh = homogeneous(X)
        self.last_inputs = Xh

        return self.W.dot(Xh.T)

    def backward(self, Y):
        input_3d = np.swapaxes(np.expand_dims(self.last_inputs, axis=0), 0, 1)
        y_t_3d = np.transpose(np.swapaxes(np.expand_dims(Y, axis=0), 0, 1), (0,2,1))
        self.grad = np.mean(y_t_3d * input_3d, axis=0)

        back_error = self.W.dot(Y.T)
        return np.delete(back_error)
    
    def __str__(self):
        return "Dense(" + str(self.input_size) + ", " + str(self.output_size) + ")"
    
class ReLU(Layer):
    def forward(self, X):
        self.last_input = X # Save inputs for backpropagation
        return np.maximum(X, 0)

    def backward(self, Y):
        return Y * np.maximum(np.sign(self.last_input), 0)

    def __str__(self):
        return "ReLU(" + str(self.input_size) + ")"

In [19]:
inputs = Dense(64, input_size=1)
x = inputs
x = ReLU()(x)
x = Dense(64)(x)
x = ReLU()(x)
x = Dense(1)(x)
outputs = x

inputs.str_chain()
print(np.dot(np.array([[2,3],[4,5]]), np.array([[2],[3]])))

[[13]
 [23]]


In [15]:
lr = 0.1

for i in range(40000):

    # Forward step
    Yp = inputs.predict(X)

    # Compute error and output gradient
    dE = Yp - Y
    E = np.mean(dE ** 2)

    if i % 1000 == 0:
        print("MSE:", E)

    if E < 0.001: # If MSE is small, stop
        print("MSE:", E)
        break

    # Backprop
    outputs.backprop(dE)

     # Update the weights (layer.W) for every trainable layer using the gradient (layer.grad)
    current_layer = inputs
    while current_layer is not None:
      if isinstance(current_layer, Dense):
        current_layer.W -= current_layer.grad * lr
      current_layer = current_layer.next

plt.scatter(X, Y)
plt.plot(Xr, inputs.predict(Xr))

NameError: name 'X' is not defined